In [1]:
# imdb  영화평 - 텍스트

In [2]:
import random
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, confusion_matrix
from torchsummary import summary
from tensorflow.keras.datasets import imdb


In [3]:
# 하이퍼 파라메터
CFG = {
    'MY_NUM':10000,             #  사전의 최대 단어수
    'MY_LEN':80,                #  각 문장의 단어수
    'MY_EMBED':30,              #  단어 임베딩의 출력 차원
    'MY_HIDDEN':100,            #  LSTM 은닉층
    'MY_EPOCHS':200,
    'SEED':41
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
(x_train,y_train),(x_test,y_test) =  imdb.load_data(num_words=CFG['MY_NUM'])  

In [7]:
x_train.shape,y_train.shape

((25000,), (25000,))

In [8]:
# 0 부정  1 긍정
print(len(x_train[0]), y_train[0])


218 1


In [9]:
for i in range(10):
  print(len(x_train[i]), y_train[i])

218 1
189 0
141 0
550 1
147 0
43 0
123 1
562 0
233 1
130 0


In [10]:
#  단어 -> 숫자
word_to_id = imdb.get_word_index()
print(f"총 단어수 : {len(word_to_id)}")
print(f"단어 the 는 {word_to_id['the']} 번째 수")

총 단어수 : 88584
단어 the 는 1 번째 수


In [11]:
# 숫자된 영화평을 문장으로 전환
# 원본.. 특수문자 3개를 처리하기위해서 각 단어에 3을 더한 값
# 숫자 - > 단어
id_to_word = { value-3:key for key,value in  word_to_id.items()}

In [12]:
temp = ''
for id in x_train[1]:
  temp += id_to_word[id]+' '
temp  

'of between impressive between respond people minutes in it henry stereotype domino go all br bring this know favourite movie often are with off previous enough films in point acting alike his movie we big this they he i br woman man too br over that grow funniest in change br did that her awarded br feel that short pieces br know that br scene 1984 you br ideas in then an plenty good just br becomes that what some about flying candy perfect br jason all corman in find as colorful to br impressive that between effort i respond to wrong dean old career to melbourne but your still than to corman but wrong few buffs think in br minutes that some to tone but instead ever in ever down as whole want something up that similarly minutes respond in to talking day roy in br raft while 10 tries not to so addressed images movie that it remember people have english can loves that this again though he br mid but film br spoiler in by it time another seemed from then people year film though any '

In [13]:
# 데이터의 길이를 통일
X_train =  pad_sequences(
    x_train,
    maxlen = CFG['MY_LEN']
)
X_test =  pad_sequences(
    x_test,
    maxlen = CFG['MY_LEN']
)

In [14]:
# 문장길이 통일 확인
for i in range(10):
  print(len(X_train[i]), y_train[i])

80 1
80 0
80 0
80 1
80 0
80 0
80 1
80 0
80 1
80 0


In [15]:
X_train.shape

(25000, 80)

In [16]:
# LSTM 을 위한 클래스 설계 - 모델
# 토치에서 모델을 상속받아서 설계.. 
# 생성자에서 각 층을 객체로 만들고.. forward에서 조립  이때 forward는 우리가 호출.. 모델에서 자동으로 training 할때 호출되는
# callback 함수
class MyLstm(nn.Module):
    def __init__(self):
      super(MyLstm, self).__init__()
      self.embedding = nn.Embedding(CFG['MY_NUM'], CFG['MY_EMBED'])
      self.lstm = nn.LSTM(CFG['MY_EMBED'], CFG['MY_HIDDEN'],batch_first=True)
      self.linear = nn.Linear(CFG['MY_HIDDEN'], 1)
      self.sigmoid = nn.Sigmoid()
    def forward(self,x):
      x = self.embedding(x)
      lstm_out,_ =  self.lstm(x)

      # lstm의 최종 은닉층 3차원 출력
      x = lstm_out[:,-1,:]
      x = self.linear(x)
      x = self.sigmoid(x)
      return x

In [17]:
model = MyLstm()
x = torch.zeros((80,1),dtype = torch.long).to(device)
model.to(device)
# summary(model,x)

MyLstm(
  (embedding): Embedding(10000, 30)
  (lstm): LSTM(30, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
criterion = nn.BCELoss().to(device)

In [19]:
# numpy -> torch.tensor
X_train = torch.LongTensor(X_train)
y_train = torch.FloatTensor(y_train)
type(X_train), type(y_train)

(torch.Tensor, torch.Tensor)

In [20]:
X_train = X_train.to(device)

In [21]:
# 데이터 셋 생성
class MyDataset(Dataset):
  def __init__(self,x,y):
    super(MyDataset, self).__init__()
    self.x = x
    self.y = y
  def __getitem__(self, index):
    return self.x[index], self.y[index]
  def __len__(self):
    return self.x.shape[0]

X_train_dataset = MyDataset(X_train,y_train)

In [22]:
# gpu를 사용할때는 미니배치개념이 중요함... 
# 데이터의 수에 따라서 할당되는 gup 메모리가 달라짐
X_train_loader = DataLoader(X_train_dataset, batch_size = 64, shuffle=True, num_workers=0)
# X_train_dataset = MyDataset(X_train,y_train)

In [23]:
# 훈련
from tqdm import tqdm
best_lowest_loss = 100
best_model = None
for epoch in range(CFG['MY_EPOCHS']):
  model.train()    
  train_loss = []
  for data,target in tqdm(X_train_loader):
    output = model(data)
    # (00000, 1)  - > (00000)
    output = torch.squeeze(output)      
    output = output.to(device)      
    target = target.to(device)
    loss = criterion(output, target)    
    train_loss.append(loss.to('cpu').detach().numpy() )
    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()          
  print(f"epoch:{epoch} mean loss:{np.mean(train_loss)} best_lowest_loss : {best_lowest_loss} ")  
  if best_lowest_loss > np.mean(train_loss):
    best_lowest_loss = np.mean(train_loss)    
    best_model = model
    print("find best model")

100%|██████████| 391/391 [00:02<00:00, 186.70it/s]


epoch:0 mean loss:0.5363430976867676 best_lowest_loss : 100 
find best model


100%|██████████| 391/391 [00:01<00:00, 241.44it/s]


epoch:1 mean loss:0.3315993845462799 best_lowest_loss : 0.5363430976867676 
find best model


100%|██████████| 391/391 [00:01<00:00, 236.69it/s]


...

epoch:197 mean loss:0.6308540105819702 best_lowest_loss : 0.05426999554038048 


100%|██████████| 391/391 [00:01<00:00, 239.20it/s]


epoch:198 mean loss:0.6315211057662964 best_lowest_loss : 0.05426999554038048 


100%|██████████| 391/391 [00:01<00:00, 235.11it/s]

epoch:199 mean loss:0.6333111524581909 best_lowest_loss : 0.05426999554038048 


In [25]:
# 평가를위해서 평가용데이터 torch변환(long)
X_test = torch.LongTensor(X_test)
y_test = torch.FloatTensor(y_test)

In [30]:
# batch 로 전환
X_test_dataset = MyDataset(X_test,y_test)
X_test_loader = DataLoader(X_test_dataset, batch_size = 64, num_workers=0)

In [40]:
threshold = 0.5
f1_score_list = []
with torch.no_grad():
  best_model.eval()  
  for data,target in tqdm(X_test_loader):
    data = data.to(device)
    # target = target.to(device)

    pred = best_model(data)
    pred = torch.squeeze(pred)
    pred = (pred > threshold).float()    
    pred = pred.to('cpu').detach().numpy()    
    f1_score_list.append(f1_score(target,pred))

100%|██████████| 391/391 [00:01<00:00, 357.00it/s]


In [41]:
np.mean(f1_score_list)

0.5865803725438976